In [ ]:
from genepy.google import gcp
import pandas as pd
from depmapomics.tracker import initTracker, SampleTracker
from depmapomics.config_prod import *
from taigapy import TaigaClient
tc = TaigaClient()
%load_ext autoreload
%autoreload 2

In [ ]:
pip install liftover

In [ ]:
from liftover import get_lifter
converter = get_lifter('hg38', 'hg19')

In [ ]:
track = SampleTracker(
        my_id=MY_ID,
        mystorage_id=MYSTORAGE_ID,
        sheetcreds=SHEETCREDS,
        refsheet_url=REFSHEET_URL,
        depmap_pv=DEPMAP_PV,
        samples_not_found_url=SAMPLES_NOT_FOUND_URL,
        samples_missing_arxspan_url=SAMPLES_MISSING_ARXSPAN_URL,
        gumbo_url=GUMBO_SHEET,
        gumbo_sheetname=GUMBO_SHEETNAME,
    )

## updating the pipeline

In [ ]:
tracker = track.read_tracker()

In [ ]:
rn = {u: v['arxspan_id'] for u,v in tracker[['arxspan_id']].iterrows()}

In [ ]:
import dalmatian as dm
wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38")

In [ ]:
samp = wm.get_samples()

In [ ]:
files = ! gsutil ls 'gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/*/call-my_vcf_to_depmap/*-maf-coding_somatic-subset.csv.gz'


In [ ]:
morefiles = ! gsutil ls 'gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/*/call-my_vcf_to_depmap/*/*-maf-coding_somatic-subset.csv.gz'

In [ ]:
len(files+morefiles)

In [ ]:
depmap_file = {i.split('/')[-1].split('-maf')[0]: i for i in files+morefiles}

In [ ]:
files = ! gsutil ls 'gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/*/call-my_vcf_to_depmap/glob-*/*.parquet'
morefiles = ! gsutil ls 'gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/*/call-my_vcf_to_depmap/*/glob-*/*.parquet'
len(files+morefiles)

In [ ]:
full_file = {sam: '["' for sam in list(set([i.split('/')[5] for i in files+morefiles]))}

In [ ]:
for f in files+morefiles:
    full_file[f.split('/')[5]]+=('", "'+f if full_file[f.split('/')[5]][-2]!='[' else f)

In [ ]:
len(depmap_file)

In [ ]:
repl = {}
for k,v in depmap_file.items():
    repl[v.split('/')[5]] = k

In [ ]:
for k in list(full_file.keys()):
    full_file[k] += ']"'

In [ ]:
set(depmap_file.keys())-set(full_file.keys())

In [ ]:
full_file['CDS-4oiBVD'] = '[ "gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/995afd45-83ae-4c8a-ac38-e4c7157f5eb9/call-my_vcf_to_depmap/glob-17f6425344ec8f0636d7ea01c552c5c4/9ac687e4eeee4116aff90689cb1902e6-0.parquet" ]'

In [ ]:
full_file['CDS-ui7Aql'] = '[ "gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/95447349-2662-48d8-81d9-fc17c984d765/call-my_vcf_to_depmap/glob-1cb8229fb23d0a4528679a2033e1adbf/d268c7595271446d9bb7e021b87b3366-0.parquet" ]'

In [ ]:
files = ! gsutil ls 'gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/*/call-open_cravat/out/*.filtered.vcf.gz.err'
morefiles = ! gsutil ls 'gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/*/call-open_cravat/*/out/*.filtered.vcf.gz.err'
len(files+morefiles)

In [ ]:
oc_err = {i.split('/')[-1].split('.')[0]: i for i in files+morefiles}

In [ ]:
oc_log = {i.split('/')[-1].split('.')[0]: i[:-3]+'log' for i in files+morefiles}

In [ ]:
df = pd.DataFrame(data={'oc_err':oc_err, 'oc_log':oc_log, 'full_file':full_file, 'depmap_file':depmap_file})
df.index='sample_id'

In [ ]:
wm.update_entity_attributes('sample', df)

In [ ]:
files = ! gsutil ls 'gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/*/call-open_cravat/out/*.filtered.vcf.gz'
morefiles = ! gsutil ls 'gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/*/call-open_cravat/*/out/*.filtered.vcf.gz'
len(files+morefiles)

In [ ]:
oc_main = {i.split('/')[-1].split('.')[0]: i for i in files+morefiles}

In [ ]:
df = pd.DataFrame(data={'oc_main_files':oc_main})
df.index.name="sample_id"            
wm.update_entity_attributes('sample', df)

In [ ]:
depmap_maf-> somatic_maf,

In [ ]:
sam = wm.get_samples()
wm.update_entity_attributes('sample', sam[~sam.depmap_maf.isna()][['depmap_maf']].rename(columns={'depmap_maf':'somatic_maf'}))

## making the files

In [ ]:
res = []
li = []
for file in files:
    if rn[file.split('/')[-1].split('-maf-')[0]] in li:
        continue
    li.append(rn[file.split('/')[-1].split('-maf-')[0]])
    maf = pd.read_csv(file)
    maf["sample_id"] = li[-1]
    res.append(maf)
res = pd.concat(res)

In [ ]:
CCLE_mutations = tc.get(name='depmap-a0ab', version=116, file='CCLE_mutations')

In [ ]:
CCLE_mutations = CCLE_mutations[CCLE_mutations.DepMap_ID.isin(li)][['Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Alternate_Allele', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript',
       'DepMap_ID', 'cDNA_Change', 'Codon_Change', 'Protein_Change',
       'isDeleterious', 'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot',
       'COSMIChsCnt', 'ExAC_AF', 'Variant_annotation', 'CGA_WES_AC']]

In [ ]:
res.to_csv('data/new.csv', index=False)

In [ ]:
res = pd.read_csv('data/new.csv', index_col=0)

## converting

In [ ]:
new = []
for (chrom,pos) in res[['chrom','pos']].values:
    d = converter[chrom][pos]
    if len(d)>0:
        new.append(list(d[0])[:2])
    else:
        new.append([chrom,pos])
    
res[['chrom','pos']] = np.array(new)

In [ ]:
len(set(res.sample_id))

In [ ]:
res

In [ ]:
res['locat'] = res.sample_id+":"+res.chrom+":"+res.pos.astype(str)
res['id'] = res.locat + ":"+res.alt

CCLE_mutations['locat'] = CCLE_mutations.DepMap_ID+":chr" + CCLE_mutations.Chromosome.astype(str) + ":" + CCLE_mutations.Start_position.astype(str)
CCLE_mutations['id'] = CCLE_mutations.locat + ":"+CCLE_mutations.Alternate_Allele

In [ ]:
CCLE_mutations = CCLE_mutations.sort_values(by='locat')
res = res.sort_values(by='locat')

## checking overlap

In [ ]:
res[res.variant_type!="SNP"].iloc[:40]

In [ ]:
CCLE_mutations.Variant_Type=='SNP'

In [ ]:
CCLE_mutations.loc[CCLE_mutations.freq.isna(), 'freq']=0

In [ ]:
a = len(set(res.locat) & set(CCLE_mutations.locat))
b = len(set(res[res.variant_type=="SNP"].locat) ^ set(CCLE_mutations[CCLE_mutations.Variant_Type=='SNP'].locat))
a / (a+b)

In [ ]:
(CCLE_mutations.freq<0.15).sum() / len(CCLE_mutations)

In [ ]:
a = len(set(res.locat) & set(CCLE_mutations.locat))
b = len(set(res[res.variant_type=="SNP"].locat) ^ set(CCLE_mutations[(~CCLE_mutations.CGA_WES_AC.isna())&(CCLE_mutations.Variant_Type=='SNP')].locat))
a / (a+b)

In [ ]:
res.ccle_deleterious.isna().sum()

In [ ]:
(~res[res.locat.isin(set(res.locat) & set(CCLE_mutations.locat))].reset_index()[['alt', 'ref_count', 'alt_count', 'popaf', 'cosmic_hotspot', 'ccle_deleterious']].ccle_deleterious.isna()).sum() / (~res.ccle_deleterious.isna()).sum()

In [ ]:
CCLE_mutations[CCLE_mutations.locat.isin(set(res.locat) & set(CCLE_mutations.locat))].reset_index()[['Alternate_Allele', 'isCOSMIChotspot', 'CGA_WES_AC', 'isDeleterious', 'locat']].isDeleterious.sum()/CCLE_mutations.isDeleterious.sum()

## issue no hotspot!!!

In [ ]:
(res.cosmic_hotspot!='Y').sum()

## comparing specific differences

In [ ]:
CCLE_mutations.shape

In [ ]:
36000/385639

In [ ]:
len(res[res.locat.isin(set(res.locat)-set(CCLE_mutations.locat))& (res.variant_type=='SNP')])

In [ ]:
198803/len(res)

10% of the mutations in CGA pipeline are not registered in the mutect2 pipeline (in CGA / freq>0.15) but

40% if the mutations in mutect2 are not in CGA

In [ ]:
CCLE_mutations['freq']=0

In [ ]:
freq = np.array(CCLE_mutations[~CCLE_mutations.CGA_WES_AC.isna()].CGA_WES_AC.str.split(':').tolist()).astype(int)
CCLE_mutations.loc[~CCLE_mutations.CGA_WES_AC.isna(),'freq'] = freq[:,0]/(freq[:,1]+freq[:,0])

In [ ]:
CCLE_mutations[CCLE_mutations.locat.isin(set(CCLE_mutations.locat)- set(res.locat))  & (~CCLE_mutations.CGA_WES_AC.isna()) & (CCLE_mutations.Variant_Type=='SNP') & (CCLE_mutations.freq>0.15)].reset_index()[['Alternate_Allele', 'isCOSMIChotspot', 'CGA_WES_AC', 'isDeleterious', 'locat']].iloc[300:]

In [ ]:
CCLE_mutations[CCLE_mutations.locat.isin(set(CCLE_mutations.locat)- set(res.locat))  & (~CCLE_mutations.CGA_WES_AC.isna()) & (CCLE_mutations.Variant_Type=='SNP') & (CCLE_mutations.freq>0.15)].reset_index()[['Alternate_Allele', 'isCOSMIChotspot', 'CGA_WES_AC', 'isDeleterious', 'locat']].iloc[300:]

1. CDS-vaC7MY: ACH-000011: locat: 11:77934713 one wes rep. het-del in hg19. gene has been removed on hg38. and merged with another one at another location (78223667). still see the deletion there but was not picked to be in the maf as it was defined as germline by mutect2.


2. CDS-wA1WJ8: ACH-002467:chr22:39,966,847: one wes rep: het del in hhg19. can be seen in exon 1 of CACNA1I: loc on hg38 is 39,570,753 and the del can be seen at 39,570,842. It has no reason to be filtered and exist under chhr22:39,966,846 according to the chain file. this is because in one it is given as AT->A instead of a T->-. dropping cases like these doesn't change the overlap amount much..

3. CDS-uMD8jg:ACH-000038:chr4:188924749 only one wes rep: het SNP in hg19. can be seen in exon of ZFP42. in hg38 itss location is chr4:187,995,771. we can see the same G-A SNP in hg38 at location chr4:188,003,595. Doesn't exist in the mutect2 output. it is noted as germline and has no reasons to be whitelisted.

In [ ]:
parq = pd.read_parquet('gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/06f7cfae-edc9-4889-adc6-627293df573e/call-my_vcf_to_depmap/attempt-3/glob-3e19b3e957fc484f883891e251a64a3d/d0d5f757a9b94940a554b73fcfc38bcc-0.parquet')

In [ ]:
v = pd.read_csv("gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/6001c090-b09a-4785-8b8a-33aa9c3a7ec6/omics_post_mutect2/3ef3dd22-043b-41e0-97d9-5307719f5d25/call-my_vcf_to_depmap/CDS-wA1WJ8-maf-coding_somatic-subset.csv.gz")
v[v.chrom=="chr22"]

In [ ]:
parq[(parq.chrom=="chr4")&(parq.pos>188_000_000)][["chrom", "pos","map_qual", "slippage", "strand_bias", "weak_evidence", "clustered_events", "base_qual", "likely_lof", "likely_driver", "is_coding", "germline", "pon", "popaf"]].

## looking at JAK2 mutations

In [ ]:
# we have more known mutations (only found not was CEBPA)
print('gene', 'new', 'old')
print('JAK2:',len(res[res.hugo_symbol=='JAK2']),len(CCLE_mutations[CCLE_mutations.Hugo_Symbol=='JAK2']))
print('EGFR:',len(res[res.hugo_symbol=='EGFR']),len(CCLE_mutations[CCLE_mutations.Hugo_Symbol=='EGFR']))
print('BRAF:',len(res[res.hugo_symbol=='BRAF']),len(CCLE_mutations[CCLE_mutations.Hugo_Symbol=='BRAF']))
print('VHL:',len(res[res.hugo_symbol=='VHL']),len(CCLE_mutations[CCLE_mutations.Hugo_Symbol=='VHL']))
print('FLT3:',len(res[res.hugo_symbol=='FLT3']),len(CCLE_mutations[CCLE_mutations.Hugo_Symbol=='FLT3']))
print('MEF2D:',len(res[res.hugo_symbol=='MEF2D']),len(CCLE_mutations[CCLE_mutations.Hugo_Symbol=='MEF2D']))
print('KRAS:',len(res[res.hugo_symbol=='KRAS']),len(CCLE_mutations[CCLE_mutations.Hugo_Symbol=='KRAS']))
print('BRCA2:',len(res[res.hugo_symbol=='BRCA2']),len(CCLE_mutations[CCLE_mutations.Hugo_Symbol=='BRCA2']))

## plotting files

In [ ]:
converter["chr22"][39570841]

In [ ]:
res[(res.sample_id=='ACH-002467') & (res.chrom=='chr22')]

In [ ]:
pd.concat([res[res.locat.isin(set(res.locat) & set(CCLE_mutations.locat))].reset_index()[['alt', 'ref_count', 'alt_count', 'popaf', 'cosmic_hotspot', 'ccle_deleterious']], CCLE_mutations[CCLE_mutations.locat.isin(set(res.locat) & set(CCLE_mutations.locat))].reset_index()[['Alternate_Allele', 'isCOSMIChotspot', 'CGA_WES_AC', 'isDeleterious', 'locat']]],axis=1)

In [ ]:
res

In [ ]:
CCLE_mutations